In [10]:
import requests
import pandas as pd
import yfinance as yf
import json
from datetime import date, timedelta,datetime
import numpy as np
import mibian
import pandas as pd
import math
import yfinance as yf
import numpy as np
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from dateutil.relativedelta import relativedelta, TH
from datetime import date, timedelta
from nsepy.derivatives import get_expiry_date
# datetime object containing current date and time
now = datetime.now()

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

symbols = ['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL',
       'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC',
       'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'INFY', 'IOC', 
       'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 'RBLBANK', 'RELIANCE', 
        'SBILIFE', 'SBIN', 'SHREECEM', 'SUNPHARMA','TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'WIPRO']
columns = ['PE.openInterest', 'PE.changeinOpenInterest', 'PE.pchangeinOpenInterest', 'PE.totalTradedVolume', 
           'PE.impliedVolatility', 'PE.lastPrice', 'PE.change', 'PE.pChange', 'PE.totalBuyQuantity', 'PE.totalSellQuantity',
           'PE.bidQty', 'PE.bidprice', 'PE.askQty', 'PE.askPrice', 'PE.underlyingValue', 'CE.strikePrice', 'CE.expiryDate',
           'CE.underlying', 'CE.identifier', 'CE.openInterest', 'CE.changeinOpenInterest', 'CE.pchangeinOpenInterest', 
           'CE.totalTradedVolume', 'CE.impliedVolatility', 'CE.lastPrice', 'CE.change', 'CE.pChange', 'CE.totalBuyQuantity', 
           'CE.totalSellQuantity', 'CE.bidQty', 'CE.bidprice', 'CE.askQty', 'CE.askPrice', 'CE.underlyingValue']


def send_msg(text):
    token = "5113096647:AAFl4Z22glYY186gavRjcpq5j0vOOsSE344"
    chat_id_1 = "1254608558" ##laveen
    chat_id_2 = "755185326" ##naman
#     chat_id_3 = "766769204" ##dhaval
    url_req_1 = "https://api.telegram.org/bot" + token + "/sendMessage" + "?chat_id=" + chat_id_1 + "&text=" + text 
    results_1 = requests.get(url_req_1)

    url_req_2 = "https://api.telegram.org/bot" + token + "/sendMessage" + "?chat_id=" + chat_id_2 + "&text=" + text 
    results_2 = requests.get(url_req_2)
    
#     url_req_3 = "https://api.telegram.org/bot" + token + "/sendMessage" + "?chat_id=" + chat_id_3 + "&text=" + text 
#     results_3 = requests.get(url_req_3)
    
    
def roundoff_underlying(df,ticker):
    data = df[df['Symbol'] == ticker]
    data.reset_index(drop=True,inplace=True)
    mylist = list(data['Strike Price'])
    mylist = list(map(float, mylist))
    myNumber = data['Underlying'][0]
    return min(mylist, key=lambda x:abs(x-myNumber))


def data():
    df = pd.DataFrame(columns = columns)
    for symbol in symbols :
        url = "https://nse-data1.p.rapidapi.com/equity_option_chain"

        querystring = {"symbol": symbol}

        headers = {
            'x-rapidapi-host': "nse-data1.p.rapidapi.com",
            'x-rapidapi-key': "c84f011115mshff0f17677ab87b0p14d794jsn12853448e7cb"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        json_dict = json.loads(response.text)
        send_msg("Successfully fetched : " + str(symbol))

        for i in range(len(json_dict['body']['records']['data'])):
            df1 = pd.json_normalize(json_dict['body']['records']['data'][i])
            df = pd.concat([df.reset_index(drop=True), df1.reset_index(drop=True)], axis = 0,ignore_index = False)
        df['PE.underlying'].fillna(value = symbol, inplace = True)
        df['PE.underlyingValue'].fillna(value = json_dict['body']['records']['underlyingValue'],inplace = True)

    ##### Column renaming ######
    data = df.copy()
    data.rename(columns = {'strikePrice':'Strike Price', 'expiryDate' : 'Expiry Date', 
                                    'CE.strikePrice' : 'ce_strikeprice', 
                                    'CE.identifier' : 'ce_identifier',  'CE.askPrice':'Call Ask Price','CE.underlyingValue':'ce_underlyingvalue',
                                    'PE.strikePrice' : 'pe_strikeprice','PE.expiryDate':'pe_expirydate',
                                    'PE.underlying':'Symbol','PE.identifier':'pe_identifier','PE.totalSellQuantity':'pe_totalsellquantity',
                                    'PE.askQty':'pe_askqty','PE.askPrice':'Put Ask Price','PE.underlyingValue':'Underlying',
                             'CE.totalSellQuantity':'ce_totalsellquantity','CE.bidQty':'ce_bidqty','CE.bidprice':'Call Bid Price','CE.askQty':'ce_askqty',
                             'CE.totalBuyQuantity':'ce_totalbuyquantity', 'Underlying' : 'Underlying', 'PE.impliedVolatility' : 'Put IV',
                        'PE.lastPrice' : "Put Last Price", 'PE.change' : 'Put Change', 'PE.pChange' : 'Put %Change', 'PE.bidprice' : 'Put Bid Price',
                        'CE.impliedVolatility' : 'Call IV', 'CE.lastPrice' : 'Call Last Price', 'CE.change' : 'Call Change', 'CE.pChange' : 'Call %Change',
                        'CE.bidprice' : 'Call Bid Price'}, inplace = True)
    # data = data[data['Symbol'].notna()]
    data.drop(['ce_strikeprice', 'CE.expiryDate', 'CE.underlying', 'ce_identifier', 'ce_underlyingvalue', 'pe_strikeprice', 
            'pe_identifier', 'pe_totalsellquantity', 'pe_askqty', 'ce_totalsellquantity',
            'ce_bidqty', 'ce_askqty', 'ce_totalbuyquantity', 'pe_strikeprice', 'pe_expirydate', 'PE.openInterest', 'PE.changeinOpenInterest',
                 'PE.pchangeinOpenInterest', 'PE.totalTradedVolume', 'CE.openInterest', 'CE.changeinOpenInterest', 'CE.pchangeinOpenInterest',
                 'CE.totalTradedVolume', 'PE.totalBuyQuantity', 'PE.bidQty'], axis = 1, inplace = True)
    ##### Final dataframe #####
    final = pd.DataFrame(columns = data.columns)
    for symbol in symbols:
        closest = roundoff_underlying(data,ticker = symbol)
        t = data[(data['Strike Price'] == closest) & (data['Symbol'] == symbol)]
        final = pd.concat([final.reset_index(drop=True), t.reset_index(drop=True)], axis = 0,ignore_index = False)
        
    return final

def rv():
    today = date.today()
    dict_rvol = {}

    for company in symbols:
        #get company data from about last months  
        send_msg("Successfully fetched : " + str(company))
        data = yf.download(company + '.NS', 
                      start = date(today.year -1 + (today.month-1)//6,(today.month-7)%12+1,1), 
                      end = date.today(), 
                      progress=False)
        data['returns'] = (np.log(data.Close /
        data.Close.shift(-1)))

        # calculate daily standard deviation of returns
        data_7 = data.tail(8)
        data_20 = data.tail(21)
        data_100 = data.tail(101)

        daily_std_7 = np.std(data_7.returns)
        daily_std_20 = np.std(data_20.returns)
        daily_std_100 = np.std(data_100.returns)

        # annualized daily standard deviation
        std_7 = daily_std_7 * 252 ** 0.5 * 100
        std_20 = daily_std_20 * 252 ** 0.5 * 100
        std_100 = daily_std_100 * 252 ** 0.5 * 100
        dict_rvol[company] = [std_7,std_20,std_100]

    df = pd.DataFrame(dict_rvol.items(), columns=['Symbol', 'Rvol'])
    final = pd.DataFrame(df['Rvol'].tolist(), columns=['R-VOL 7','R-VOL 20', 'R-VOL 100'])
    final['Symbol'] = df['Symbol']
    final['Date'] = date.today()
    
    return final
send_msg("Script Started at : " + str(dt_string))
send_msg("Fetching data from RapidAPI NSE")

data_df = data()
send_msg("Calculating Realised Volatility")
rv_df = rv()


data_df = data_df.sort_values(by=['Expiry Date'])
# data_df = data_df.groupby("Symbol").first()
data_df['Straddle'] = (data_df['Put Ask Price'] + data_df['Put Ask Price'])/2 + \
                      (data_df['Call Ask Price'] + data_df['Call Ask Price'])/2
data_df['Straddle %'] = (data_df['Straddle'] / data_df['Underlying']) * 100
data_final = data_df.drop(['Put Last Price', 'Put Change', 'Put %Change', 'Put Bid Price','Put Ask Price',
             'Call Last Price', 'Call Change', 'Call %Change', 'Call Bid Price', 'Call Ask Price'],axis = 1)
data_final.reset_index(inplace = True)
final = pd.merge(left = data_final, right = rv_df, left_on = 'Symbol', right_on = 'Symbol')
final = final.reindex(columns=['Symbol','Date','Expiry Date','Underlying','Strike Price','Straddle',
                              'Straddle %','Call IV','Put IV','R-VOL 7','R-VOL 20','R-VOL 100'])


send_msg("Calculating Edge for Today")
################################################################################################################################
comp_list = ['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 
        'BANDHANBNK', 'BHARTIARTL','BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 
        'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC','HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
        'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'INFY', 'IOC', 'ITC', 
        'JSWSTEEL', 'KOTAKBANK', 'LT', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 
        'RBLBANK', 'RELIANCE', 'SBILIFE', 'SBIN', 'SHREECEM', 'SUNPHARMA','TATACONSUM', 'TATAMOTORS',
        'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'WIPRO']

def get_last_thurs(year,month):
    dt= date(year,month ,1)
    cmon = month 
    t=dt
    for i in range(1, 6):
        t = dt + relativedelta(weekday=TH(i))
        if t.month != cmon:
        # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
            t = t + relativedelta(weekday=TH(-2))
            break
    return t

dict_rvol={}
for comp in comp_list:
    dict_rvol[comp]=0
    
ZZZ= []

today = date.today()
cmon = today.month
t=date.today()
for i in range(1, 6):
    t = today + relativedelta(weekday=TH(i))
    if t.month != cmon:
        # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
        t = t + relativedelta(weekday=TH(-2))
        break
days_to_expiry = t.day -today.day        
print('expiry on : ',end='')
print(t)
if(t.day >today.day+7 ):
    print('expiry in this month')
    
    for i in range(1,today.month):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year,month=i)
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
#             print(days_considered)
            close = close[~np.isnan(close)]
#             print(close.shape[0])
#             print(close[-1])
            
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print(len(log))
#             print('log and std')
#             print(log)
            std=np.std(log)
#             print(std)
            rvol = std* np.sqrt(252)
            dict_rvol[comp]+=rvol/12
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
            
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
    
    
    
    for i in range(today.month,13):
        #getting last thurs of that month
#         set_thurs = get_expiry_date(year=today.year-1,month=i)
#         list_thurs = []
#         for _ in set_thurs:
#             list_thurs.append(_)
#         list_thurs.sort()
#         thurs=list_thurs[-1]
        thurs = get_last_thurs(year=today.year-1,month=i)
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year -1  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
#             print(close[-1])
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
            close = close[~np.isnan(close)]
#             print(days_considered)
#             print(close.shape[0])
          
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print('log and std')
#             print(log)
            std=np.std(log)
            rvol = std* np.sqrt(252)
#             print(std)
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
            

elif( t.day<today.day):
    days_to_expiry=35
    print('expiry in next month')
    for i in range(1,today.month+1):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year,month=i)
        print('expiry on '+ str(thurs))
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            
            #print(close.shape[0])
             
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
#             print(days_considered)
#             print(close.shape[0])
            close = close[~np.isnan(close)]
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print('log and std')
#             print(log)
#             print(len(log))
            std=np.std(log)
#             print(std)
            rvol = std* np.sqrt(252)
            
            
               
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
            
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
    
           
    
    for i in range(today.month+1,13):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year-1,month=i)
        
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year -1  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
            
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
#             print(days_considered)
            close = close[-days_considered:]
            #print(days_considered)
#             print(close.shape[0])
          #np.log10(close/close.shift())
            close = close[~np.isnan(close)]
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
            
            std=np.std(log)
            rvol = std* np.sqrt(252)
            
            
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
                
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')


df = pd.DataFrame(list(dict_rvol.items()),columns = ['Symbol', 'Hist_RV'])
df['Hist_RV'] = round(df['Hist_RV'] * 100,2)
left_df = final.copy()
test = pd.merge(left = left_df, right = df, left_on = 'Symbol', right_on = 'Symbol')



def straddle(underlying_price,call_strike,volatility,straddle_price):
    interest_rate = 5
    
    today = date.today()
    cmon = today.month

    for i in range(1, 6):
        t = today + relativedelta(weekday=TH(i))
        if t.month != cmon:
            # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
            t = t + relativedelta(weekday=TH(-2))
            break
            
    days_to_expiry = t.day -today.day
    p = mibian.BS([underlying_price, call_strike, interest_rate, days_to_expiry], volatility = volatility)

    edge = ((straddle_price - (p.callPrice + p.putPrice)) * 100) / underlying_price
    edge = round(edge,2)
#     print(underlying_price,call_strike,interest_rate,days_to_expiry,volatility,straddle_price,edge)
    return edge



test['Edge'] = test.apply(lambda row : straddle(row['Underlying'],row['Strike Price'],row['Hist_RV'],row['Straddle']), axis = 1)
# test['R-VOL 7'] = round(test['R-VOL 7'],2)
# test['R-VOL 20'] = round(test['R-VOL 20'],2)
# test['R-VOL 100'] = round(test['R-VOL 100'],2)
# test['Straddle'] = round(test['Straddle'],2)
decimals = 2    
test['R-VOL 7'] = test['R-VOL 7'].apply(lambda x: round(x, decimals))
test['R-VOL 20'] = test['R-VOL 20'].apply(lambda x: round(x, decimals))
test['R-VOL 100'] = test['R-VOL 100'].apply(lambda x: round(x, decimals))
test['Straddle'] = test['Straddle'].apply(lambda x: round(x, decimals))
send_msg("________________________________Successfully Calculated_____________________________________")

test.to_csv("screener.csv")

expiry on : 2022-03-31
expiry in this month
expiry on 2022-01-27
getting data from ADANIPORTS
0.019961128618496842 ADANIPORTS rvol 
getting data from ASIANPAINT
0.018457363616155537 ASIANPAINT rvol 
getting data from AUBANK
0.029532197382204903 AUBANK rvol 
getting data from AXISBANK
0.02412236850561145 AXISBANK rvol 
getting data from BAJAJ-AUTO
0.02061233317396274 BAJAJ-AUTO rvol 
getting data from BAJAJFINSV
0.02800360342829904 BAJAJFINSV rvol 
getting data from BAJFINANCE
0.025256256256769014 BAJFINANCE rvol 
getting data from BANDHANBNK
0.02284698934522429 BANDHANBNK rvol 
getting data from BHARTIARTL
0.01995827664902285 BHARTIARTL rvol 
getting data from BPCL
0.016523110416392497 BPCL rvol 
getting data from BRITANNIA
0.013230094190132059 BRITANNIA rvol 
getting data from CIPLA
0.014214373138055292 CIPLA rvol 
getting data from COALINDIA
0.021788190115244816 COALINDIA rvol 
getting data from DIVISLAB
0.01822309259740394 DIVISLAB rvol 
getting data from DRREDDY
0.01181245328887378

0.0567162371617451 HINDALCO rvol 
getting data from HINDUNILVR
0.03850360772270296 HINDUNILVR rvol 
getting data from ICICIBANK
0.042513808397020736 ICICIBANK rvol 
getting data from IDFCFIRSTB
0.058989299633970774 IDFCFIRSTB rvol 
getting data from INDUSINDBK
0.05794109113385881 INDUSINDBK rvol 
getting data from INFY
0.046572219418438685 INFY rvol 
getting data from IOC
0.0439821830858022 IOC rvol 
getting data from ITC
0.04104842953682952 ITC rvol 
getting data from JSWSTEEL
0.0674479381526225 JSWSTEEL rvol 
getting data from KOTAKBANK
0.05273417456257288 KOTAKBANK rvol 
getting data from LT
0.04809672483364376 LT rvol 
getting data from MARUTI
0.053563912627735195 MARUTI rvol 
getting data from NESTLEIND
0.030022908244972963 NESTLEIND rvol 
getting data from NTPC
0.04588896441104522 NTPC rvol 
getting data from ONGC
0.05684186794001392 ONGC rvol 
getting data from PNB
0.06256556790102216 PNB rvol 
getting data from POWERGRID
0.045517230173203076 POWERGRID rvol 
getting data from RB

0.10319566281196899 TATASTEEL rvol 
getting data from TCS
0.05359846710551546 TCS rvol 
getting data from TECHM
0.06525279444346115 TECHM rvol 
getting data from TITAN
0.06497421148367666 TITAN rvol 
getting data from ULTRACEMCO
0.06657170745911374 ULTRACEMCO rvol 
getting data from UPL
0.08736602912675859 UPL rvol 
getting data from WIPRO
0.07919302101005556 WIPRO rvol 
expiry on 2021-06-24
getting data from ADANIPORTS
0.11781204343190811 ADANIPORTS rvol 
getting data from ASIANPAINT
0.07177671954120739 ASIANPAINT rvol 
getting data from AUBANK
0.12045391488203834 AUBANK rvol 
getting data from AXISBANK
0.08242779907396887 AXISBANK rvol 
getting data from BAJAJ-AUTO
0.07510712207321038 BAJAJ-AUTO rvol 
getting data from BAJAJFINSV
0.0982474534362485 BAJAJFINSV rvol 
getting data from BAJFINANCE
0.10398152442110414 BAJFINANCE rvol 
getting data from BANDHANBNK
0.09666087875472094 BANDHANBNK rvol 
getting data from BHARTIARTL
0.06555224356526419 BHARTIARTL rvol 
getting data from BPCL
0

0.08853053434893826 HCLTECH rvol 
getting data from HDFC
0.093040859578307 HDFC rvol 
getting data from HDFCBANK
0.08072549217721192 HDFCBANK rvol 
getting data from HDFCLIFE
0.08014011227131501 HDFCLIFE rvol 
getting data from HEROMOTOCO
0.08359450807640857 HEROMOTOCO rvol 
getting data from HINDALCO
0.12329903629805897 HINDALCO rvol 
getting data from HINDUNILVR
0.07048223181546727 HINDUNILVR rvol 
getting data from ICICIBANK
0.08823834863205093 ICICIBANK rvol 
getting data from IDFCFIRSTB
0.12731162325627557 IDFCFIRSTB rvol 
getting data from INDUSINDBK
0.1174734597529648 INDUSINDBK rvol 
getting data from INFY
0.07356089891846651 INFY rvol 
getting data from IOC
0.0854220161468464 IOC rvol 
getting data from ITC
0.07332082831138788 ITC rvol 
getting data from JSWSTEEL
0.13000308097225927 JSWSTEEL rvol 
getting data from KOTAKBANK
0.09298272140825398 KOTAKBANK rvol 
getting data from LT
0.09137346431764029 LT rvol 
getting data from MARUTI
0.09001145129487877 MARUTI rvol 
getting da

0.11111855736159193 SHREECEM rvol 
getting data from SUNPHARMA
0.08625838011571416 SUNPHARMA rvol 
getting data from TATACONSUM
0.10460695861314533 TATACONSUM rvol 
getting data from TATAMOTORS
0.1774816787660355 TATAMOTORS rvol 
getting data from TATASTEEL
0.16935954349054916 TATASTEEL rvol 
getting data from TCS
0.08460299291043583 TCS rvol 
getting data from TECHM
0.11176477040866195 TECHM rvol 
getting data from TITAN
0.11287509142427934 TITAN rvol 
getting data from ULTRACEMCO
0.09734795146131167 ULTRACEMCO rvol 
getting data from UPL
0.13080380036441033 UPL rvol 
getting data from WIPRO
0.12483872489314932 WIPRO rvol 
expiry on 2021-11-25
getting data from ADANIPORTS
0.16586980603408735 ADANIPORTS rvol 
getting data from ASIANPAINT
0.11437619335349958 ASIANPAINT rvol 
getting data from AUBANK
0.18003949311217196 AUBANK rvol 
getting data from AXISBANK
0.11993448240058839 AXISBANK rvol 
getting data from BAJAJ-AUTO
0.10856305688115803 BAJAJ-AUTO rvol 
getting data from BAJAJFINSV


In [8]:
final.head()

,Symbol,Date,Expiry Date,Underlying,Strike Price,Straddle,Straddle %,Call IV,Put IV,R-VOL 7,R-VOL 20,R-VOL 100
0,WIPRO,2022-03-09,26-May-2022,587.7,590.0,NaN,NaN,0,NaN,22.992209,34.487033,30.392772
1,WIPRO,2022-03-09,28-Apr-2022,587.7,590.0,32.1,5.46197,33.85,0,22.992209,34.487033,30.392772
2,WIPRO,2022-03-09,31-Mar-2022,587.7,590.0,38.45,6.542454,0,37.42,22.992209,34.487033,30.392772
3,ICICIBANK,2022-03-09,26-May-2022,670.0,670.0,NaN,NaN,0,NaN,45.996113,39.061295,31.709119
4,ICICIBANK,2022-03-09,28-Apr-2022,670.0,670.0,69.9,10.432836,0,0,45.996113,39.061295,31.709119


In [9]:
left_df.head()

,Symbol,Date,Expiry Date,Underlying,Strike Price,Straddle,Straddle %,Call IV,Put IV,R-VOL 7,R-VOL 20,R-VOL 100
0,WIPRO,2022-03-09,26-May-2022,587.7,590.0,NaN,NaN,0,NaN,22.992209,34.487033,30.392772
1,WIPRO,2022-03-09,28-Apr-2022,587.7,590.0,32.1,5.46197,33.85,0,22.992209,34.487033,30.392772
2,WIPRO,2022-03-09,31-Mar-2022,587.7,590.0,38.45,6.542454,0,37.42,22.992209,34.487033,30.392772
3,ICICIBANK,2022-03-09,26-May-2022,670.0,670.0,NaN,NaN,0,NaN,45.996113,39.061295,31.709119
4,ICICIBANK,2022-03-09,28-Apr-2022,670.0,670.0,69.9,10.432836,0,0,45.996113,39.061295,31.709119


In [10]:
df.head()

,Symbol,Hist_RV
0,ADANIPORTS,17.73
1,ASIANPAINT,12.28
2,AUBANK,19.64
3,AXISBANK,12.85
4,BAJAJ-AUTO,11.64


In [11]:
test = pd.merge(left = left_df, right = df, left_on = 'Symbol', right_on = 'Symbol')

In [7]:
test.round(1)

,Symbol,Date,Expiry Date,Underlying,Strike Price,Straddle,Straddle %,Call IV,Put IV,R-VOL 7,R-VOL 20,R-VOL 100,Hist_RV,Edge
0,FEDERALBNK,2022-03-11,26-May-2022,94.6,95.0,NaN,NaN,0,NaN,50.5,50.3,38.0,16.9,NaN
1,FEDERALBNK,2022-03-11,28-Apr-2022,94.6,95.0,0.0,0.0,49.16,0,50.5,50.3,38.0,16.9,-3.2
2,FEDERALBNK,2022-03-11,31-Mar-2022,94.6,95.0,7.95,8.403805,41.77,45.59,50.5,50.3,38.0,16.9,5.2
3,SHREECEM,2022-03-11,26-May-2022,22060.1,22000.0,NaN,NaN,NaN,0,36.6,45.1,30.0,13.0,NaN
4,SHREECEM,2022-03-11,28-Apr-2022,22060.1,22000.0,0.0,0.0,0,0,36.6,45.1,30.0,13.0,-2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,HCLTECH,2022-03-11,31-Mar-2022,1190.9,1190.0,66.95,5.621799,22.85,29.69,16.2,26.7,28.4,12.3,3.3
120,HDFC,2022-03-11,28-Apr-2022,2232.55,2240.0,0.0,0.0,0,26.42,40.1,38.5,27.3,12.0,-2.2
121,HDFC,2022-03-11,31-Mar-2022,2232.55,2240.0,135.25,6.058095,28.25,32.14,40.1,38.5,27.3,12.0,3.8
122,AUBANK,2022-03-11,28-Apr-2022,1156.0,1160.0,0.0,0.0,0,0,55.3,53.8,43.8,19.7,-3.7


In [4]:
test.head()

,Symbol,Date,Expiry Date,Underlying,Strike Price,Straddle,Straddle %,Call IV,Put IV,R-VOL 7,R-VOL 20,R-VOL 100,Hist_RV,Edge
0,FEDERALBNK,2022-03-11,26-May-2022,94.6,95.0,NaN,NaN,0,NaN,50.528604,50.294742,37.961686,16.93,NaN
1,FEDERALBNK,2022-03-11,28-Apr-2022,94.6,95.0,0.0,0.0,49.16,0,50.528604,50.294742,37.961686,16.93,-3.17
2,FEDERALBNK,2022-03-11,31-Mar-2022,94.6,95.0,7.95,8.403805,41.77,45.59,50.528604,50.294742,37.961686,16.93,5.24
3,SHREECEM,2022-03-11,26-May-2022,22060.1,22000.0,NaN,NaN,NaN,0,36.567845,45.124823,30.021792,13.01,NaN
4,SHREECEM,2022-03-11,28-Apr-2022,22060.1,22000.0,0.0,0.0,0,0,36.567845,45.124823,30.021792,13.01,-2.46


In [15]:
def straddle(underlying_price,call_strike,volatility,straddle_price):
    interest_rate = 5
    
    today = date.today()
    cmon = today.month

    for i in range(1, 6):
        t = today + relativedelta(weekday=TH(i))
        if t.month != cmon:
            # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
            t = t + relativedelta(weekday=TH(-2))
            break
            
    days_to_expiry = t.day -today.day
    p = mibian.BS([underlying_price, call_strike, interest_rate, days_to_expiry], volatility = volatility)

    edge = ((straddle_price - (p.callPrice + p.putPrice)) * 100) / underlying_price
    edge = round(edge,2)
#     print(underlying_price,call_strike,interest_rate,days_to_expiry,volatility,straddle_price,edge)
    return edge



test['Edge'] = test.apply(lambda row : straddle(row['Underlying'],row['Strike Price'],row['Hist_RV'],row['Straddle']), axis = 1)
# test['R-VOL 7'] = round(test['R-VOL 7'],2)
# test['R-VOL 20'] = round(test['R-VOL 20'],2)
# test['R-VOL 100'] = round(test['R-VOL 100'],2)
# test['Straddle'] = round(test['Straddle'],2)
send_msg("________________________________Successfully Calculated_____________________________________")

test.to_csv("screener.csv")

In [ ]:
# 